In [16]:
# Warning control
import warnings
import os

warnings.filterwarnings("ignore")

from crewai import Agent, Task, Crew
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = os.getenv("OPENAI_MODEL_NAME")

## agent


In [17]:
support_agent = Agent(
    role="Senior Support Representative",
    goal="Be the most friendly and helpful " "support representative in your team",
    backstory=(
        "You work at crewAI (https://crewai.com) and "
        " are now working on providing "
        "support to {customer}, a super important customer "
        " for your company."
        "You need to make sure that you provide the best support!"
        "Make sure to provide full complete answers, "
        " and make no assumptions."
    ),
    allow_delegation=False,
    verbose=True,
)

-- in the assurance one we are saying the agent can delegate the work back to the previous agent

-- see the allow_delegation is True by default

-- but when to delegate and when not to delegate?

         -- for this, we are solely dependent on the llms ability to trust it


In [18]:
support_quality_assurance_agent = Agent(
    role="Support Quality Assurance Specialist",
    goal="Get recognition for providing the "
    "best support quality assurance in your team",
    backstory=(
        "You work at crewAI (https://crewai.com) and "
        "are now working with your team "
        "on a request from {customer} ensuring that "
        "the support representative is "
        "providing the best support possible.\n"
        "You need to make sure that the support representative "
        "is providing full"
        "complete answers, and make no assumptions."
    ),
    verbose=True,
)

- **Role Playing**: Both agents have been given a role, goal and backstory.

- **Focus**: Both agents have been prompted to get into the character of the roles they are playing.

- **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together


## Learning:

        -- add a QA in the multi agentic systemt so that it does QA stuff

        -- it is found that with the QA stuff, the agentic stuff works much better


## Tools, Guardrails and Memory

### Tools

        -- some of the crew_ai tools like scrape the internet or do some things with git repo and all


In [19]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

## serperdevtool : integrates wirh server: external service to search on Google

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- `Some` ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

In [20]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

## creating task
- for support engineer answer
- QAing the answer given by the support agent

In [21]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
        "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
        "Make sure to use everything you know "
        "to provide the best support possible."
        "You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
        "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
        "leaving no questions unanswered, and maintain a helpful and friendly "
        "tone throughout."
    ),
    tools=[docs_scrape_tool],
    agent=support_agent,
)

In [22]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
        "high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
        "thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
        "ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
        "relevant feedback and improvements.\n"
        "Don't be too formal, we are a chill and cool company "
        "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)

## Notice:
-  we did not provide the tools for the QA or the quality assurance review
- its main task is just to look at the output given by the previous support agent and use that answer to verify all

## Creating the CREW

In [23]:
crew = Crew(
    agents=[support_agent, support_quality_assurance_agent],
    tasks=[inquiry_resolution, quality_assurance_review],
    verbose=2,
    memory=True,
)

2025-04-12 17:25:08,574 - 8234502208 - __init__.py-__init__:535 - WARNING: Overriding of current TracerProvider is not allowed


## Demo run

In [24]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
    "and kicking it off, specifically "
    "how can I add memory to my crew? "
    "Can you provide guidance?",
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
I need to provide thorough guidance to DeepLearningAI on how to add memory to their Crew and kick it off. I must ensure that my response is detailed, informative, and helpful to meet their needs.

Action: Read website content
Action Input: {"url": "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"} 

Introduction - CrewAICrewAI home pageSearch CrewAI docscrewAIInc/crewAIcrewAIInc/crewAISearch...NavigationGet StartedIntroductionGet StartedExampl

In [25]:
from IPython.display import Markdown

Markdown(result)

Dear Andrew Ng from DeepLearningAI,

Thank you for reaching out with your important question about setting up a Crew and adding memory to it. I understand the significance of this task for your organization, and I am here to provide you with detailed guidance on how to achieve this.

To add memory to your Crew and ensure optimal performance, follow these step-by-step instructions:

1. Identify the specific Crew that requires additional memory.
2. Determine the type of memory required for the Crew (e.g., RAM, storage).
3. Purchase the appropriate memory module that is compatible with the Crew's existing hardware.
4. Power off the Crew and disconnect all cables to ensure safety.
5. Open the Crew casing to access the internal components.
6. Locate the memory slots on the Crew's motherboard.
7. Carefully insert the new memory module into the empty memory slot, ensuring it is properly aligned.
8. Apply gentle pressure to secure the memory module in place.
9. Close the Crew casing and reattach all cables.
10. Power on the Crew and verify that the memory has been successfully added.

By following these steps, you can effectively add memory to a Crew and enhance its capabilities. The process is crucial for improving the performance of your AI agents and ensuring smooth operations.

For more detailed information and examples, you can refer to the CrewAI Documentation at https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/.

If you have any further questions or need additional assistance, please feel free to reach out. I am here to support you every step of the way.

Best regards,

[Your Name]
Senior Support Representative
crewAI